In [1]:
import os
import dask
from time import sleep
import utils.dirutils as dd
from dask.distributed import Client
import cupy as cp
import numpy as np
from dask_jobqueue import SLURMCluster
# import scipy as sp
# from numba import njit
# import computation2_L as cc
import computation2_L as cc
import utils.dirutils as dd

/opt/conda/envs/RAPIDS/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
path = dd.base()
daskpath=os.path.join(path, "dask-worker-space-can-be-deleted")
try: 
    os.mkdir(daskpath) 
except OSError as error: 
    print(error)		 

# proc * threads = 18(cores) 
cluster = SLURMCluster(
    log_directory=daskpath
)   

print(cluster.job_script())
cluster.scale(jobs=1) # 1 nodes
sleep(15)
client = Client(cluster)
client.amm.start()

[Errno 17] File exists: '/home/quojinhao/Data/dask-worker-space-can-be-deleted'


/opt/conda/envs/RAPIDS/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36619 instead
  warnings.warn(


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /home/quojinhao/Data/dask-worker-space-can-be-deleted/dask-worker-%J.err
#SBATCH -o /home/quojinhao/Data/dask-worker-space-can-be-deleted/dask-worker-%J.out
#SBATCH -p gpu
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=233G
#SBATCH -t 144:00:00

/opt/conda/envs/RAPIDS/bin/python -m distributed.cli.dask_worker tcp://192.168.0.254:34693 --nthreads 1 --nprocs 4 --memory-limit 58.21GiB --name dummy-name --nanny --death-timeout 90 --local-directory /scratch/$USER/$SLURM_JOB_ID --resources gpu=1 --protocol tcp://



In [ ]:
client.status

In [4]:
# for L
P1 = np.arange(1,3)
P2 = np.arange(3,5)
P3 = np.arange(5,7)
P4 = np.arange(7,9)
fileformat = "L{}5um_1e-5_normalized.h5"
nj = 1000000
stepj = nj//10

In [ ]:
# for G
nj = 400000
stepj = 50000
fileformat = "G{}5um_1e-5_L1.h5"
G1 = np.array([5,2])
G2 = np.array([1,0.8])
G3 = np.array([0.5,0.1])
G4 = np.array([0.01,0.001])

In [3]:
def setup_gpu_worker(i):
    sleep(5)
    cp.cuda.Device(i).use()
    sleep(5)
with dask.annotate(resources={'gpu': 1}): # must included. proc--one-to-one--gpu
    setupGPUs = [ client.submit( setup_gpu_worker, i ) for i in range(4) ]

In [ ]:
with dask.annotate(resources={'gpu': 1}): # must included. proc--one-to-one--gpu
    futures = [ client.submit( cc.computation, P, nj, stepj, fileformat ) for P in [P1, P2, P3, P4] ]

In [ ]:
with dask.annotate(resources={'gpu': 1}): # must included. proc--one-to-one--gpu
    futures = [ client.submit( cc.computation, G, nj, stepj, fileformat ) for G in [G1, G2, G3, G4] ]

In [ ]:
client.cancel(future)
# client.cancel( futures )

In [ ]:
client.close()
cluster.close()

In [ ]:
client.restart()